In [22]:
import pandas as pd
import numpy as np
from scipy.signal import butter,filtfilt
import plotly.graph_objects as go

In [37]:
df_acc = pd.read_csv('MDS_Scenario_2_nils_1 2022-05-23 10-48-47/Accelerometer.csv')
# Drop the first 7 readings, because they're missing in the gyroscope data
df_acc.drop([i for i in range(0,7)], axis=0, inplace=True)
# Drop the old index
df_acc.reset_index(inplace=True)
df_acc.drop(labels='index', axis=1, inplace=True)

df_gyr = pd.read_csv('MDS_Scenario_2_nils_1 2022-05-23 10-48-47/Gyroscope.csv')
# concat the dataframes
df = pd.concat([df_acc, df_gyr], axis=1)
# Drop the second Time(s) column
df = df.loc[:,~df.columns.duplicated()]

length of acc:  11342
length of gyr:  11342


In [42]:
readings_np = df.to_numpy()

In [46]:
def butter_lowpass_filter(data):
    cutoff = 2
    fs = 200
    order = 2
    normal_cutoff = cutoff / 100
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

In [68]:
filtered_readings_np = np.apply_along_axis(butter_lowpass_filter, 0, readings_np[:, 1:])

In [69]:
filtered_readings_np = np.c_[readings_np[:, 0], filtered_readings_np]

In [73]:
# Filter the data, and plot both the original and filtered signals.
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 1],
            line =  dict(shape =  'spline' ),
            name = 'x signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 2],
            line =  dict(shape =  'spline' ),
            name = 'y signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 3],
            line =  dict(shape =  'spline' ),
            name = 'z signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 0],
            line =  dict(shape =  'spline' ),
            name = 'time signal')
            )
fig.show()